# 汽车里程数回归预测实验

本实验以开源的auto-mpg数据集为基础，基于MindSpore深度学习库应用全连接神经网络进行汽车里程数预测。

## 1、实验目的

- 掌握全连接神经网络的原理。
- 了解如何使用MindSpore进行简单的回归模型的训练。
- 了解如何使用MindSpore进行简单的回归模型的预测。

## 2、全连接神经网络的原理介绍

浅层神经网络相比单层网络的差别在于隐藏层有多个神经节点，这就使得其可以处理“多输入多输出”的复杂问题。每一层的每一个节点都与上下层节点全部连接，这种神经网络称作全连接网络。

![image.png](attachment:image.png)

### 2.1 正向传播

$$z^{[1]} = \left\lgroup\begin{matrix}z_{1}^{[1]} \cr z_{2}^{[2]} \cr z_{3}^{[3]}\end{matrix}\right\rgroup = \left\lgroup\begin{matrix}w_{1}^{[1]T}\cdot x + b_{1}^{[1]} \cr w_{2}^{[2]T}\cdot x + b_{2}^{[2]} \cr w_{3}^{[3]T}\cdot x + b_{3}^{[3]}\end{matrix}\right\rgroup = \left\lgroup\begin{matrix}w_{1}^{[1]T}\cdot x \cr w_{2}^{[2]T}\cdot x \cr w_{3}^{[3]T}\cdot x  \end{matrix}\right\rgroup + b^{[1]} = W^{[1]}X + b^{[1]}$$



$$a^{[1]} = \left\lgroup\begin{matrix}a_{1}^{[1]} \cr a_{2}^{[2]} \cr a_{3}^{[3]}\end{matrix}\right\rgroup = \left\lgroup\begin{matrix}t\left(z_{1}^{[1]}\right) \cr t\left(z_{2}^{[2]}\right) \cr t\left(z_{3}^{[3]}\right)\end{matrix}\right\rgroup = t\left\lgroup\begin{matrix}\left(z_{1}^{[1]}\right) \cr \left(z_{2}^{[2]}\right) \cr \left(z_{3}^{[3]}\right) \end{matrix}\right\rgroup = t\left(z^{[1]}\right)$$

- 上角标中括号用于区分不同层
- 下角标数字表示神经元节点的映射关系
- 一个神经元节点包含上一层节点数$w$和$b$和下一层节点数$z$

### 2.2 反向传播

梯度下降法：

$$W = W - \alpha  \frac{\partial L}{\partial W}$$
$$b = b - \alpha  \frac{\partial L}{\partial b}$$

式中$W$和$b$为模型的权重参数，$L$为模型定义的损失函数，$\alpha$为超参数学习率；以上两式可以理解为：通过损失函数$L$对权重参数$W$和$b$进行求取导数，并利用导数乘以学习率对原来的$W$和$b$进行更新。

## 3、实验环境

- MindSpore 2.0（MindSpore版本会定期更新，本指导也会定期刷新，与版本配套）；
- 本案例支持win_x86和Linux系统，CPU/Ascend均可运行。
- 如果在本地运行此实验，请参考《MindSpore环境搭建实验手册》在本地安装MindSpore。

## 4、数据处理

### 4.1数据准备

这个数据集来自卡内基梅隆大学维护的StatLib库。1983年美国统计协会博览会使用了该数据集。这个数据集是对StatLib库中提供的数据集稍加修改的版本。根据Ross Quinlan(1993)在预测属性“mpg”中的使用，删除了 8 个原始实例，因为它们的“mpg”属性值未知。原始数据集在“auto-mpg.data-original”文件中。

该数据集共计9个特征，398个样本，用于回归任务。

![image.png](attachment:image.png)

In [ ]:
from download import download

# 下载汽车里程auto-mpg数据集
url = " https://ascend-professional-construction-dataset.obs.cn-north-4.myhuaweicloud.com:443/deep-learning/auto-mpg.zip"  
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|███████████████████████████| 6.68k/6.68k [00:00<00:00, 745kB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


### 4.2数据加载

- 导入模型库：

os模块主要用于对系统路径和文件进行处理。Numpy模块主要用于数据的基本运算操作。Matplotlib模块主要用于画图。MindSpore相关模块主要用于搭建网络、调用优化器、读取数据集和将数据集处理成网络的标准输入格式。

In [ ]:
#导入相关依赖库
import  os
import csv
import time
import numpy as np
import pandas as pd  #版本采用1.3.0
from matplotlib import pyplot as plt

import mindspore as ms
import mindspore.dataset as ds
from mindspore.dataset import GeneratorDataset
import mindspore.dataset.transforms as C
import mindspore.dataset.vision as CV

from mindspore import nn, Tensor, ops
from mindspore.train import Model
from mindspore.train import Accuracy, MAE, MSE
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor

ms.set_context(mode=ms.GRAPH_MODE, device_target='CPU')  # device_target支持"Ascend"、"CPU"。

- 查看数据格式

In [ ]:
#加载数据集
with open('./auto-mpg.data') as csv_file:
    data = list(csv.reader(csv_file, delimiter=','))
print(data[20:40]) # 打印部分数据

[['25.0   4   110.0      87.00      2672.      17.5   70  2\t"peugeot 504"'], ['24.0   4   107.0      90.00      2430.      14.5   70  2\t"audi 100 ls"'], ['25.0   4   104.0      95.00      2375.      17.5   70  2\t"saab 99e"'], ['26.0   4   121.0      113.0      2234.      12.5   70  2\t"bmw 2002"'], ['21.0   6   199.0      90.00      2648.      15.0   70  1\t"amc gremlin"'], ['10.0   8   360.0      215.0      4615.      14.0   70  1\t"ford f250"'], ['10.0   8   307.0      200.0      4376.      15.0   70  1\t"chevy c20"'], ['11.0   8   318.0      210.0      4382.      13.5   70  1\t"dodge d200"'], ['9.0    8   304.0      193.0      4732.      18.5   70  1\t"hi 1200d"'], ['27.0   4   97.00      88.00      2130.      14.5   71  3\t"datsun pl510"'], ['28.0   4   140.0      90.00      2264.      15.5   71  1\t"chevrolet vega 2300"'], ['25.0   4   113.0      95.00      2228.      14.0   71  3\t"toyota corona"'], ['25.0   4   98.00      ?          2046.      19.0   71  1\t"ford pinto"'], ['

- 利用pandas模块读取数据

Pandas模块是一个处理表格类数据非常有效的模块。

In [ ]:
#使用pandas读取数据
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
#遇到？换成nan，忽略\t之后的内容，已空格作为分隔符。
raw_data = pd.read_csv('./auto-mpg.data', names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

data = raw_data.copy()

#查看数据形状
data.shape

(398, 8)

- 数据预处理

对于数据集中的空值，我们要在建模前进行处理。此处空值的数据较少，我们直接进行删除。特征Origin代表着车辆的归属区域信息，此处总共分为三种，欧洲，美国，日本，我们需要对此特征进行one-hot编码。

In [ ]:
#对于数据集中的空值，我们要在建模前进行处理。此处空值的数据较少，我们直接进行删除。
#清洗空数据
data = data.dropna()
data.tail()
#Pandas库提供了简单的数据集统计信息，我们可直接调用函数describe()进行查看。
#查看训练数据集的结构
origin = data.pop('Origin')
data_labels = data.pop('MPG')
train_stats = data.describe()
train_stats = train_stats.transpose()
train_stats
#归一化数据
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_data = norm(data)
# 将MPG放回归一化后的数据中
normed_data['MPG'] = data_labels
# 离散特征处理
# 特征Origin代表着车辆的归属区域信息，此处总共分为三种，欧洲，美国，日本，我们需要对此特征进行one-hot编码。
# 对origin属性进行one-hot编码
normed_data['USA'] = (origin == 1)*1.0
normed_data['Europe'] = (origin == 2)*1.0
normed_data['Japan'] = (origin == 3)*1.0

模型训练需要区分特征值与目标值，也就是我们常说的X值与Y值，此处MPG为Y值，其余的特征为X值。

在模型构建的时候，我们一般需要准备两份数据，一份训练集数据用于模型的训练构建，一份测试集用于模型的评估优化。创建比例0.8，用于分割训练集和验证集，80%的数据用于模型训练，20%的数据用于模型验证。并把数据集处理成模型所需的数据格式。

In [ ]:
#将数据集按照4：1划分成训练集和测试集
train_dataset = normed_data.sample(frac=0.8,random_state=0)
test_dataset = normed_data.drop(train_dataset.index)

#模型训练需要区分特征值与目标值，也就是我们常说的X值与Y值，此处MPG为Y值，其余的特征为X值。
#将目标值和特征分开
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

X_train, Y_train = np.array(train_dataset, dtype=np.float32), np.array(train_labels, dtype=np.float32)
X_test, Y_test = np.array(test_dataset, dtype=np.float32), np.array(test_labels, dtype=np.float32)

#查看数据集尺寸
print('训练数据x尺寸：',X_train.shape)
print('训练数据y尺寸：',Y_train.shape)
print('测试数据x尺寸：',X_test.shape)
print('测试数据y尺寸：',Y_test.shape)

训练数据x尺寸： (314, 9)
训练数据y尺寸： (314,)
测试数据x尺寸： (78, 9)
测试数据y尺寸： (78,)


In [ ]:
# Iterable object as input source
class Iterable:
    def __init__(self, X, Y):
        self._data = X
        self._label = Y[:, np.newaxis]

    def __getitem__(self, index):
        return self._data[index], self._label[index]

    def __len__(self):
        return len(self._data)

data = Iterable(X_train, Y_train)
dataset_train = GeneratorDataset(source=data, column_names=["data", "label"])
data = Iterable(X_test, Y_test)
dataset_test = GeneratorDataset(source=data, column_names=["data", "label"])

## 5、模型构建

汽车油耗里程数据集准备完成，接下来我们就需要构建训练模型，本实验采用的是全连接神经网络算法，所以我们首先需要建立初始化的神经网络。nn.cell能够用来组成网络模型;模型共包含3个全连接，采用Relu当做激活函数。

In [ ]:
# 定义网络
class Regression_car(nn.Cell):
    def __init__(self):
        super(Regression_car, self).__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.fc1 = nn.Dense(9,64, activation='relu')
        self.fc2 = nn.Dense(64,64, activation='relu')
        self.fc3 = nn.Dense(64,1)
        
        
    def construct(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

## 6、模型训练与测试

指定模型所需的损失函数、评估指标、优化器等参数。回归问题，损失函数采用均方误差MSE。将创建好的网络、损失函数、评估指标、优化器等参数装入模型中对模型进行训练。需要逐步打印出MAE、MSE的值。

In [ ]:
#定义网络，损失函数，评估指标  优化器
network = Regression_car()
net_loss = nn.MSELoss()
net_opt = nn.RMSProp(network.trainable_params(), 0.001)

# 定义用于训练的train_loop函数。
def train_loop(model, dataset, loss_fn, optimizer):
    # 定义正向计算函数
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss

    # 定义微分函数，使用mindspore.value_and_grad获得微分函数grad_fn,输出loss和梯度。
    # 由于是对模型参数求导,grad_position 配置为None，传入可训练参数。
    grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

    # 定义 one-step training函数
    def train_step(data, label):
        loss, grads = grad_fn(data, label)
        optimizer(grads)
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        data, label = ms.Tensor(data[:, np.newaxis].T, ms.float32), ms.Tensor(label[:, np.newaxis], ms.float32)
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

# 定义用于测试的test_loop函数。
def test_loop(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        data, label = ms.Tensor(data[:, np.newaxis].T, ms.float32), ms.Tensor(label[:, np.newaxis], ms.float32)
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f" Avg loss: {test_loss:>8f} \n")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(network, dataset_train, net_loss, net_opt)
    test_loop(network, dataset_test, net_loss)
print("Done!")

Epoch 1
-------------------------------
loss: 400.001221  [  0/314]
loss: 224.835861  [100/314]
loss: 36.852665  [200/314]
loss: 10.433537  [300/314]
 Avg loss: 15.366340 

Epoch 2
-------------------------------
loss: 78.916695  [  0/314]
loss: 92.176308  [100/314]
loss: 1.867100  [200/314]
loss: 0.489709  [300/314]
 Avg loss: 8.757651 

Epoch 3
-------------------------------
loss: 8.841584  [  0/314]
loss: 53.447109  [100/314]
loss: 6.799405  [200/314]
loss: 182.811768  [300/314]
 Avg loss: 9.886613 

Epoch 4
-------------------------------
loss: 2.432492  [  0/314]
loss: 0.008578  [100/314]
loss: 1.231788  [200/314]
loss: 0.000276  [300/314]
 Avg loss: 7.996487 

Epoch 5
-------------------------------
loss: 1.316351  [  0/314]
loss: 3.636306  [100/314]
loss: 34.566517  [200/314]
loss: 1.199968  [300/314]
 Avg loss: 7.481086 

Epoch 6
-------------------------------
loss: 0.954388  [  0/314]
loss: 3.801863  [100/314]
loss: 0.069054  [200/314]
loss: 7.224389  [300/314]
 Avg loss: 9.